## download data

In [ ]:
import pandas as pd 
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class mpdb:
    pdb_id_chain_list = []
    atom_coord = []
df = pd.read_csv(r"/content/drive/My Drive/nrlist_3.178_4.0A.csv")
df = pd.DataFrame(df.values, columns=["number", "pdb_id", "details"])
df.head()
pdb_id_data = pd.DataFrame(df, columns=["pdb_id"])

id_list = []
chain_list = []
print(len(pdb_id_data))
for i in range(0, len(pdb_id_data)):
    pdb_id, _, chain_name = pdb_id_data.loc[i][0].split('+')[0].split('|')
    id_list.append(pdb_id)
    chain_list.append(chain_name)
    # for j in range(1,len(pdb_id_data.loc[i][1].split(','))):
    #   pdb_id, _, chain_name = pdb_id_data.loc[i][1].split(",")[j].split('+')[0].split("|")
    #   id_list.append(pdb_id)
    #   chain_list.append(chain_name)

    
mpdb.pdb_id_chain_list = list(zip(id_list, chain_list))
print("number of pdb_id & chain_id : ", len(mpdb.pdb_id_chain_list))
##inja chon ye seri zanjire haye mokhtalef marboot b ye pdb id hastan niazi nist chand bar ye pdb_id ro d konim!
mpdb.pdb_id_chain_list = tuple(set(mpdb.pdb_id_chain_list))
print("number of pdb_id & chain_list (unique): ", len(mpdb.pdb_id_chain_list))

print("\n")
print("number of pdb_id: ", len(id_list))

id_list = np.unique(id_list)
print(r"number of pdb_id(unique)--->(we need this for download files): ", len(id_list))

2083
number of pdb_id & chain_id :  2083
number of pdb_id & chain_list (unique):  2083


number of pdb_id:  2083
number of pdb_id(unique)--->(we need this for download files):  1838


In [ ]:
flag=0
for i in range(0,len(pdb_id_data)):
    if(pdb_id_data.loc[i][0].split('+')[0].split('|')[1] != "1"):
        print ("warning : one data model isnt \"1\"")
        print(pdb_id_data.loc[i][0].split('+')[0].split('|'))
        flag=1
if (flag==0):
    print("its ok")
s=0
import requests
for id in id_list[0:len(id_list)]:
  url="https://files.rcsb.org/download/"+str(id)+".cif"
  r = requests.get(url, allow_redirects=True)
  open(r"/content/drive/My Drive/new_data/{}.cif".format(str(id)), 'wb').write(r.content)
  s=s+1
print(s)

['4R3I', '0', 'B']
1838


##import and install

In [ ]:
!pip install prody
!pip install pylab
!pip install biopython

     |████████████████████████████████| 6.3MB 5.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.3MB 38.6MB/s 
  Created wheel for prody: filename=ProDy-2.0-cp37-cp37m-linux_x86_64.whl size=6653554 sha256=1693a8d3666d377a62790cd8257888664e2f66618a2d359f9d52f0819c8614a2
  Stored in directory: /root/.cache/pip/wheels/e2/be/52/a879e8adf713065f8f11e15c56073b311e82ecc85bbeb8b6c4
Successfully built prody
ERROR: Could not find a version that satisfies the requirement pylab (from versions: none)
ERROR: No matching distribution found for pylab


In [ ]:
from prody import *
from pylab import *
%matplotlib inline
confProDy(auto_show=False)
confProDy(auto_secondary=True)
import pandas as pd 
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from math import sqrt
import math
import re
import Bio.PDB
parser = Bio.PDB.MMCIFParser()
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("default")
from tqdm import tqdm_notebook
from tqdm import tqdm
from numpy import pi
RAD2DEG = 180 / pi

@> ProDy is configured: auto_show=False
@> ProDy is configured: auto_secondary=True


In [ ]:
import urllib3
http = urllib3.PoolManager()
import pandas as pd
from bs4 import BeautifulSoup
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##def ha

In [ ]:
def calcDihedral(atoms1, atoms2, atoms3, atoms4, radian=False):
    """Returns the dihedral angle between atoms in degrees."""

    if not isinstance(atoms1, Bio.PDB.Atom.Atom):
        raise TypeError('atoms1 must be an Atomic instance')
    if not isinstance(atoms2, Bio.PDB.Atom.Atom):
        raise TypeError('atoms2 must be an Atomic instance')
    if not isinstance(atoms3, Bio.PDB.Atom.Atom):
        raise TypeError('atoms3 must be an Atomic instance')
    if not isinstance(atoms4, Bio.PDB.Atom.Atom):
        raise TypeError('atoms4 must be an Atomic instance')
    # if not (atoms1.numAtoms() == atoms2.numAtoms() ==
    #         atoms3.numAtoms() == atoms4.numAtoms()):
    #     raise ValueError('all arguments must have same number of atoms')

    return getDihedral(atoms1.get_coord(), atoms2.get_coord(),
                       atoms3.get_coord(), atoms4.get_coord(), radian)

In [ ]:
def getDihedral(coords1, coords2, coords3, coords4, radian=False):
    """Returns the dihedral angle in degrees unless ``radian=True``."""

    a1 = coords2 - coords1
    a2 = coords3 - coords2
    a3 = coords4 - coords3

    v1 = cross(a1, a2)
    v1 = v1 / (v1 * v1).sum(-1)**0.5
    v2 = cross(a2, a3)
    v2 = v2 / (v2 * v2).sum(-1)**0.5
    porm = sign((v1 * a3).sum(-1))
    rad = arccos((v1*v2).sum(-1) / ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5)
    if not porm == 0:
        rad = rad * porm
    if radian:
        return rad
    else:
        return rad * RAD2DEG

In [ ]:
def new_download_secondary_structure(pdb_id, chain):
  r = http.request(
        'POST',
        'http://dssr.x3dna.org/',
        fields={'pdbid': pdb_id , 'luxfile': '(binary)' ,
                'coords': json.dumps(content_dict)})
  soup = BeautifulSoup(r.data.decode('utf-8'), 'lxml')
  a=soup.get_text()
  f=a.split("****************************************************************************")
  for i in range(len(f)):
    for j in f[i].split("\n"):
      if j=='Secondary structures in dot-bracket notation (dbn) as a whole and per chain':
        sec_struct=f[i].split(pdb_id.lower() + '-'+chain)[1].split('\n')[2]

  return(sec_struct)

In [ ]:
def find_number(text, c):
    return re.findall(r'%s(\-?\d+)' % c, text) 

In [ ]:
def distance_finder(one,two) :      
  [x1,y1,z1] = one # first coordinates     
  [x2,y2,z2] = two # second coordinates      
  dist = (((x2-x1)**2)+((y2-y1)**2)+((z2-z1)**2))**(1/2)     
  return dist

##main

In [ ]:
class mpdb:
    pdb_id_chain_list = []
    atom_coord = []
df = pd.read_csv(r"/content/drive/My Drive/nrlist_3.178_4.0A.csv")
 
df = pd.DataFrame(df.values, columns=["number", "pdb_id", "details"])
df.head()
pdb_id_data = pd.DataFrame(df, columns=["pdb_id", "details"])
 
id_list = []
chain_list = []
for i in range(0, len(pdb_id_data)):
    pdb_id, _, chain_name = pdb_id_data.loc[i][0].split('+')[0].split('|')
    id_list.append(pdb_id)
    chain_list.append(chain_name)
    
mpdb.pdb_id_chain_list = list(zip(id_list, chain_list))
print("number of pdb_id & chain_id : ", len(mpdb.pdb_id_chain_list))
##inja chon ye seri zanjire haye mokhtalef marboot b ye pdb id hastan niazi nist chand bar ye pdb_id ro d konim!
mpdb.pdb_id_chain_list = tuple(set(mpdb.pdb_id_chain_list))
print("number of pdb_id & chain_list (unique): ", len(mpdb.pdb_id_chain_list))
 
print("\n")
print("number of pdb_id: ", len(id_list))
 
id_list = np.unique(id_list)
print(r"number of pdb_id(unique)--->(we need this for download files): ", len(id_list))

number of pdb_id & chain_id :  2083
number of pdb_id & chain_list (unique):  2083


number of pdb_id:  2083
number of pdb_id(unique)--->(we need this for download files):  1838


In [ ]:
##felan ba 50 ta run mikonam k edame bedam vali hanuz sakhtar dovom nadaram
incomplete_file = 0
longest_length = 0
longest_length_id = 0
sum_length = 0
n = 0
broken_data=[]
print("\n")
#len(mpdb.pdb_id_chain_list)
for i in tqdm(range(len(mpdb.pdb_id_chain_list))):
    flag = True
    pdb_id, pdb_chainname = mpdb.pdb_id_chain_list[i]
    pdb_id = pdb_id.upper()
    print(pdb_id, pdb_chainname)
    structure = parser.get_structure(str(pdb_id), r"/content/drive/My Drive/new_data/" + str(pdb_id) + ".cif")
    chain = [c for c in list(structure.get_chains()) if c.id == str(pdb_chainname)][0]
    if len(chain) == 1:
        print(pdb_id)
        continue

    x_pos = []
    y_pos = []
    z_pos = []
    incomplete_atom_coord = 0
    residue_name_list = []
    residue_number_list = []
    res_count = 0
    O5_list = []
    C5_list = []
    C4_list = []
    C3_list = []
    O3_list = []
    P_list = []
    p_coord = []
    c5_coord = []
    o5_coord = []
    c4_coord=[]
    c3_coord = []
    o3_coord = []

    res1= 0
    dihedral = []
    sin_dihedral = []
    cos_dihedral = []
    scnd_struct = []

    # scnd_struct.append(download_secnd_struct(str(pdb_id),pdb_chainname))
    #inja k res1 neveshtam baraye kenar gozoshtane residue aval hast.
    for res in chain.get_residues():
        flagp,flagc5,flago5,flagc4,flagc3,flago3=False , False ,False ,False ,False ,False 
        if res.get_resname() not in "ACGU":
          continue
        if res1 == 0:
          res1 = res1 + 1
          continue
  
        res_count += 1
        atom_count = 0
        # p_e = 0
        residue_name_list.append(res.get_resname())
        residue_number_list.append(find_number(res.__repr__(), "resseq="))
        xs = 0
        ys = 0
        zs = 0
        for atom in res.get_atoms():
            ##inja age coordination nadashte bashe un atom 999 baghi mimone adadesh!!:)
            x = 999
            y = 999
            z = 999
            
            if atom.get_name() == "P" :
              if atom.is_disordered() == 0 :
                P_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagp = True
                xs += x
                ys += y
                zs += z
                p_coord.append([x,y,z])

                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C5'":
              if atom.is_disordered() == 0 :
                C5_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc5 = True
                xs += x
                ys += y
                zs += z
                c5_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "O5'":
              if atom.is_disordered() == 0 :
                O5_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flago5 = True
                xs += x
                ys += y
                zs += z
                o5_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C4'":
              if atom.is_disordered() == 0 :
                C4_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc4 = True
                xs += x
                ys += y
                zs += z
                c4_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C3'":
              if atom.is_disordered() == 0 :
                C3_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc3 = True
                xs += x
                ys += y
                zs += z
                c3_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "O3'":
              if atom.is_disordered() == 0 :
                O3_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flago3 = True
                xs += x
                ys += y
                zs += z
                o3_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1

            
            

        if flagp == False or flagc5==False or flago5 == False or flagc4 == False or flagc3 == False or flago3 == False:
          flag = False
          broken_data.append([pdb_id,pdb_chainname,res_count])
          # print("this file doesnt have all atom! pdbid: {} , chain: {} , res:{}".format(pdb_id,pdb_chainname,res_count), g )
        else:
          x_pos.append(xs / atom_count)
          y_pos.append(ys / atom_count)
          z_pos.append(zs / atom_count)

    #     edited
    if (res_count <= 49):
        
        incomplete_file += 1
        print("incomplete file : {}_{}".format(pdb_id, pdb_chainname))

    else:
      if len(C5_list) == len(O5_list) == len(C4_list)== len(C3_list) == len(O3_list)==len(P_list) != 0:
        if flag == True:
          for i in range(len(C5_list)):
            if i == 0:
              alpha = None
              beta= calcDihedral(P_list[i], O5_list[i],C5_list[i],C4_list[i])
              gamma= calcDihedral(O5_list[i], C5_list[i],C4_list[i],C3_list[i])
              delta = calcDihedral(C5_list[i],C4_list[i],C3_list[i] , O3_list[i])
              epsilon = calcDihedral(C4_list[i],C3_list[i] , O3_list[i] , P_list[i+1])
              zeta = calcDihedral(C3_list[i] , O3_list[i] , P_list[i+1] , O5_list[i+1])
              sin_dihedral.append([None, math.sin(beta) , math.sin(gamma ),math.sin( delta) ,math.sin( epsilon) ,math.sin( zeta)])
              cos_dihedral.append([ None , math.cos(beta) , math.cos(gamma ),math.cos( delta) ,math.cos( epsilon) ,math.cos( zeta)])
            elif i == len(C5_list)-1:
              alpha = calcDihedral(O3_list[i-1], P_list[i],O5_list[i],C5_list[i])
              beta= calcDihedral(P_list[i], O5_list[i],C5_list[i],C4_list[i])
              gamma= calcDihedral(O5_list[i], C5_list[i],C4_list[i],C3_list[i])
              delta = calcDihedral(C5_list[i],C4_list[i],C3_list[i] , O3_list[i])
              epsilon = None
              zeta = None
              sin_dihedral.append([math.sin(alpha) , math.sin(beta) , math.sin(gamma ),math.sin( delta) , None , None])
              cos_dihedral.append([math.cos(alpha) , math.cos(beta) , math.cos(gamma ),math.cos( delta) , None , None])

            else:
              alpha = calcDihedral(O3_list[i], P_list[i],O5_list[i],C5_list[i])
              beta= calcDihedral(P_list[i], O5_list[i],C5_list[i],C4_list[i])
              gamma= calcDihedral(O5_list[i], C5_list[i],C4_list[i],C3_list[i])
              delta = calcDihedral(C5_list[i],C4_list[i],C3_list[i] , O3_list[i])
              epsilon = calcDihedral(C4_list[i],C3_list[i] , O3_list[i] , P_list[i+1])
              zeta = calcDihedral(C3_list[i] , O3_list[i] , P_list[i+1] , O5_list[i+1])
              sin_dihedral.append([math.sin(alpha) , math.sin(beta) , math.sin(gamma ),math.sin( delta) ,math.sin( epsilon) ,math.sin( zeta)])
              cos_dihedral.append([math.cos(alpha) , math.cos(beta) , math.cos(gamma ),math.cos( delta) ,math.cos( epsilon) ,math.cos( zeta)])
            dihedral.append([alpha ,beta , gamma, delta , epsilon  , zeta])


##bayad ina ro berizam tou list vaa baad ham tou file zakhire konam
          if (len(x_pos) == len(y_pos) == len(z_pos)):
            if incomplete_atom_coord == 0:
                  if longest_length <= len(x_pos):
                      longest_length = len(x_pos)
                      longest_length_id = (pdb_id, pdb_chainname)

                  sum_length += len(x_pos)
                  n += 1

                  with open(r"/content/drive/My Drive/new_txt_data/{}_{}.txt".format(pdb_id, pdb_chainname), 'w') as f:
                      print("residue\t", ",".join(str(x) for x in residue_name_list), file=f)
                      print("residue_number\t", ",".join(str(x) for x in residue_number_list), file=f)
                      print("dihedral\t", ",".join(str(x) for x in dihedral), file=f)
                      print("sin_dihedral\t", ",".join(str(x)for x in sin_dihedral), file=f)
                      print("cos_dihedral\t", ",".join(str(x) for x in cos_dihedral), file=f)
                      print("mean_of 6_x_pos\t", ",".join(str(x) for x in x_pos), file=f)
                      print("mean_of 6_y_pos\t", ",".join(str(x) for x in y_pos), file=f)
                      print("mean_of 6_z_pos\t", ",".join(str(x) for x in z_pos), file=f)
                      print("secondary_structure\t", ",".join(str(x) for x in scnd_struct), file=f)
                      print("p_coords\t", ",".join(str(x) for x in p_coord), file=f)
                      print("c5_coords\t", ",".join(str(x) for x in c5_coord), file=f)
                      print("o5_coords\t", ",".join(str(x) for x in o5_coord), file=f)
                      print("c4_coords\t", ",".join(str(x) for x in c4_coord), file=f)
                      print("c3_coords\t", ",".join(str(x) for x in c3_coord), file=f)
                      print("o3_coords\t", ",".join(str(x) for x in o3_coord), file=f)
                      print("distance_nucltd  :" ,file = f)
                      for i in range(len(c4_coord)):
                        print("distances for c4_coord : {}\t".format(residue_number_list[i]) ,file = f)
                        for j in range(len(c4_coord)):
                          print("with {}".format(residue_number_list[i]),distance_finder(c4_coord[i],c4_coord[j]), file= f)


            if incomplete_atom_coord == 1:
              incomplete_file += 1
              print("incomplete file(incomplete coordinates) : {}_{}".format(pdb_id, pdb_chainname))

          else:
            incomplete_file += 1
            print("incomplete file(incomplete coordinates) : {}_{}".format(pdb_id, pdb_chainname))

      
print("number of incomplete file = ", incomplete_file)
print("longest length = ", longest_length, "\t", longest_length_id)
print("mean of length = ", sum_length / n)
print("number of broken data = " , len(broken_data))



  0%|          | 0/2083 [00:00<?, ?it/s]



6ZMI CC




  0%|          | 1/2083 [00:15<9:12:41, 15.93s/it]

422D A




  0%|          | 2/2083 [00:16<6:30:54, 11.27s/it]

...

incomplete file : 1DQH_B
4Z7K C




100%|█████████▉| 2082/2083 [3:08:00<00:01,  1.64s/it]

incomplete file : 4Z7K_C
6MSF R




100%|██████████| 2083/2083 [3:08:00<00:00,  5.42s/it]

incomplete file : 6MSF_R
number of incomplete file =  1476
longest length =  3772 	 ('6ZMI', 'L5')
mean of length =  361.2918215613383
number of broken data =  776


In [ ]:
 print("tedade rna haye nahaii:",n)

NameError: ignored

In [ ]:
##baraye check kardane brocken data k age mishe beshkounameshoun(b nazaram felan vaght talaf kardane!baad ageshod mikonam k b data ezafe beshe!)
incomplete_file = 0
longest_length = 0
longest_length_id = 0
sum_length = 0
n = 0
broken_data=[]
print("\n")
#len(mpdb.pdb_id_chain_list)
for i in tqdm(range(len(mpdb.pdb_id_chain_list))):
    flag = True
    pdb_id, pdb_chainname = mpdb.pdb_id_chain_list[i]
    pdb_id = pdb_id.upper()
    print(pdb_id, pdb_chainname)
    structure = parser.get_structure(str(pdb_id), r"/content/drive/My Drive/new_data/" + str(pdb_id) + ".cif")
    chain = [c for c in list(structure.get_chains()) if c.id == str(pdb_chainname)][0]
    if len(chain) == 1:
        print(pdb_id)
        continue

    x_pos = []
    y_pos = []
    z_pos = []
    incomplete_atom_coord = 0
    residue_name_list = []
    residue_number_list = []
    res_count = 0
    O5_list = []
    C5_list = []
    C4_list = []
    C3_list = []
    O3_list = []
    P_list = []
    p_coord = []
    c5_coord = []
    o5_coord = []
    c4_coord=[]
    c3_coord = []
    o3_coord = []

    res1= 0
    dihedral = []
    sin_dihedral = []
    cos_dihedral = []
    scnd_struct = []

    # scnd_struct.append(download_secnd_struct(str(pdb_id),pdb_chainname))
    #inja k res1 neveshtam baraye kenar gozoshtane residue aval hast.
    for res in chain.get_residues():
        flagp,flagc5,flago5,flagc4,flagc3,flago3=False , False ,False ,False ,False ,False 
        if res.get_resname() not in "ACGU":
          continue
        if res1 == 0:
          res1 = res1 + 1
          continue
  
        res_count += 1
        atom_count = 0
        # p_e = 0
        residue_name_list.append(res.get_resname())
        residue_number_list.append(find_number(res.__repr__(), "resseq="))
        xs = 0
        ys = 0
        zs = 0
        for atom in res.get_atoms():
            ##inja age coordination nadashte bashe un atom 999 baghi mimone adadesh!!:)
            x = 999
            y = 999
            z = 999
            
            if atom.get_name() == "P" :
              if atom.is_disordered() == 0 :
                P_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagp = True
                xs += x
                ys += y
                zs += z
                p_coord.append([x,y,z])

                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C5'":
              if atom.is_disordered() == 0 :
                C5_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc5 = True
                xs += x
                ys += y
                zs += z
                c5_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "O5'":
              if atom.is_disordered() == 0 :
                O5_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flago5 = True
                xs += x
                ys += y
                zs += z
                o5_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C4'":
              if atom.is_disordered() == 0 :
                C4_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc4 = True
                xs += x
                ys += y
                zs += z
                c4_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "C3'":
              if atom.is_disordered() == 0 :
                C3_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flagc3 = True
                xs += x
                ys += y
                zs += z
                c3_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1
            elif atom.get_name() == "O3'":
              if atom.is_disordered() == 0 :
                O3_list.append(atom)
                x, y, z = atom.get_coord()
                atom_count += 1
                flago3 = True
                xs += x
                ys += y
                zs += z
                o3_coord.append([x,y,z])
                if x == 999 or y == 999 or z == 999:
                  incomplete_atom_coord = 1

            
            

        if flagp == False or flagc5==False or flago5 == False or flagc4 == False or flagc3 == False or flago3 == False:
          flag = False
          broken_data.append([pdb_id,pdb_chainname,res_count])

##append secondary structure to text file

In [ ]:
import os
import json
import urllib3
import pandas as pd
http = urllib3.PoolManager()
from bs4 import BeautifulSoup

In [ ]:
pdb_id_chain = os.listdir(r"/content/drive/My Drive/data")

FileNotFoundError: ignored

In [ ]:
pdb_id_chain[0].split('.')[0].split('_')

['5EMO', 'F']

In [ ]:
pid_chain=[]
for i in range(len(pdb_id_chain)):
  pid_chain.append(pdb_id_chain[i].split('.')[0].split('_'))

In [ ]:
pdb_id=[]
for i in range(len(pdb_id_chain)):
  pdb_id.append(pdb_id_chain[i].split('.')[0].split('_')[0])

In [ ]:
content_dict = {'modelSelection': 'FIRST','basePairAnalyzer': 'DSSR','_analyzeHelices': 'on',
              'nonCanonicalHandling': 'ONLY_VISUALIZE',
              '_removeIsolated': 'on' ,
              'pseudoknotFinder': 'HYBRID' ,
              'structuralElementsIdentificationMode': 'PSEUDOKNOTS_AS_PAIRED_RESIDUES'
              ,'secondaryStructureDrawer': 'VARNA_MODIFIED'}


def get_uuid_from_analyze1(pdb_id):
  r = http.request(
      'POST',
      'http://rnapdbee.cs.put.poznan.pl/analyze',
      fields={'inputType': 'PDB','source': pdb_id+'.pdb', 'serverResource': 'fetch '+pdb_id,
              'content': json.dumps(content_dict)})
  soup = BeautifulSoup(r.data.decode('utf-8'), 'lxml')
  a = soup.find_all('input', attrs={'name':'uuid', 'class':'uuid'})
  # try:
  return a[0]['value'], r


def get_uuid_from_analyze2(pdb_id):
  r = http.request(
      'POST',
      'http://rnapdbee.cs.put.poznan.pl/analyze',
      fields={'inputType': 'MMCIF','source': pdb_id+'.pdb', 'serverResource': 'fetch '+pdb_id,
              'content': json.dumps(content_dict)})
  soup = BeautifulSoup(r.data.decode('utf-8'), 'lxml')
  a = soup.find_all('input', attrs={'name':'uuid', 'class':'uuid'})
  print(a)
  return a[0]['value'], r

def download_zip_file(pdb_id):
  # try:
  res, rq = get_uuid_from_analyze1(pdb_id)
  # except IndexError:
      # res, rq = get_uuid_from_analyze2(pdb_id)
  sess = dict(rq.headers)['Set-Cookie'].split(';')[0]
  req = http.request('POST','http://rnapdbee.cs.put.poznan.pl/download', fields={'uuid': res ,'isDotBracket': 'on' },
                  headers={'Cookie':'_ga=GA1.3.934375168.1591791412; _gid=GA1.3.1441166492.1594305168; '+sess+'; _gat=1,',
                          'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"})
  if req.status == 200:
    f = open(r"/content/drive/My Drive/secc_struct_RNApdbee/{}.zip".format(pdb_id), 'w+b')
    f.write(req.data)
    f.close()
  return 1


def download_zip_file2(pdb_id):
  # try:
  res, rq = get_uuid_from_analyze2(pdb_id)
  # except IndexError:
      # res, rq = get_uuid_from_analyze2(pdb_id)
  sess = dict(rq.headers)['Set-Cookie'].split(';')[0]
  req = http.request('POST','http://rnapdbee.cs.put.poznan.pl/download', fields={'uuid': res ,'isDotBracket': 'on' },
                  headers={'Cookie':'_ga=GA1.3.934375168.1591791412; _gid=GA1.3.1441166492.1594305168; '+sess+'; _gat=1,',
                          'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"})
  if req.status == 200:
    f = open(r"/content/drive/My Drive/secc_struct_RNApdbee/{}.zip".format(pdb_id), 'w+b')
    f.write(req.data)
    f.close()
  return 1


In [ ]:
for i in pdb_id:
  print(i)
  try:
    print(1)
    download_zip_file(i)
  except :
    print(2)
    download_zip_file2(i)
  finally:
    print(3)
    download_zip_file2(i)

5EMO
1
3
[]


IndexError: ignored

In [ ]:
def get_uuid_from_analyze2(pdb_id):
  r = http.request(
      'POST',
      'http://rnapdbee.cs.put.poznan.pl/analyze',
      fields={'inputType': 'MMCIF','source': pdb_id+'.pdb', 'serverResource': 'fetch '+pdb_id,
              'content': json.dumps(content_dict)})
  soup = BeautifulSoup(r.data.decode('utf-8'), 'lxml')
  print(soup)
  a = soup.find_all('input', attrs={'name':'uuid', 'class':'uuid'})
  print(a)
  return a[0]['value'], r